In [1]:
#import necessary libraries
import requests
import pandas as pd

from bs4 import BeautifulSoup
import re

from datetime import date
import pandas as pd
import json

### **create csv file: dim_match_summary**

In [2]:
#store all the json objects(match records) in a list
with open('t20_json_files/t20_wc_match_results.json') as f:
  data = json.load(f)
  all_json_records = []
  for json_obj in data[0]["matchSummary"]:
    all_json_records.append(json_obj)
    
# print(all_json_records)

In [3]:
#create the dataframe "df_match_info"

df_match_info = pd.DataFrame(all_json_records)
df_match_info.head()

,Team_1,Team_2,Winner,Margin,Ground,Match_Date,Scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [4]:
#rename "scorecard" with "match_id"

df_match_info.rename({'Scorecard': 'match_id'}, axis = 1, inplace = True)

In [5]:
#create a dictionary for match_ids [This is helpful to give match_ids for other fact table]

match_ids_dict = {}
for index, row in df_match_info.iterrows():
  val1 = row['Team_1'] + ' Vs ' + row['Team_2']
  val2 = row['Team_2'] + ' Vs ' + row['Team_1']
  match_id = row['match_id']

  match_ids_dict[val1] = match_id
  match_ids_dict[val2] = match_id

In [6]:
#interchange the position of columns 

df_match_info = df_match_info[['match_id', 'Team_1', 'Team_2', 'Match_Date', 'Ground', 'Winner', 'Margin']]
df_match_info.head()

,match_id,Team_1,Team_2,Match_Date,Ground,Winner,Margin
0,T20I # 1823,Namibia,Sri Lanka,"Oct 16, 2022",Geelong,Namibia,55 runs
1,T20I # 1825,Netherlands,U.A.E.,"Oct 16, 2022",Geelong,Netherlands,3 wickets
2,T20I # 1826,Scotland,West Indies,"Oct 17, 2022",Hobart,Scotland,42 runs
3,T20I # 1828,Ireland,Zimbabwe,"Oct 17, 2022",Hobart,Zimbabwe,31 runs
4,T20I # 1830,Namibia,Netherlands,"Oct 18, 2022",Geelong,Netherlands,5 wickets


In [7]:
df_match_info.tail()

,match_id,Team_1,Team_2,Match_Date,Ground,Winner,Margin
40,T20I # 1872,Bangladesh,Pakistan,"Nov 6, 2022",Adelaide,Pakistan,5 wickets
41,T20I # 1873,India,Zimbabwe,"Nov 6, 2022",Melbourne,India,71 runs
42,T20I # 1877,New Zealand,Pakistan,"Nov 9, 2022",Sydney,Pakistan,7 wickets
43,T20I # 1878,England,India,"Nov 10, 2022",Adelaide,England,10 wickets
44,T20I # 1879,England,Pakistan,"Nov 13, 2022",Melbourne,England,5 wickets


In [9]:
#save the data to csv file

df_match_info.to_csv('t20_csv_files/dim_match_summary.csv', index = False)

### **create csv file: fact_batting_summary**

In [32]:
#store all the json objects in a list
with open('t20_json_files/t20_wc_batting_summary.json') as f:
  data = json.load(f)
  all_json_records = []
  for json_obj in data:
    all_json_records.extend(json_obj["battingSummary"])
  
# print(all_json_records)

In [34]:
#create the dataframe "df_batsman_sum"

df_batsman_sum = pd.DataFrame(all_json_records)
print(df_batsman_sum.shape)
df_batsman_sum.head()

(689, 10)


,match,team_innings,batting_pos,batsman_name,dismissal,run,ball,4s,6s,SR
0,Namibia Vs Sri Lanka,Namibia Innings,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00
1,Namibia Vs Sri Lanka,Namibia Innings,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00
2,Namibia Vs Sri Lanka,Namibia Innings,3,Jan Nicol Loftie-Eaton,c â€ Mendis b Karunaratne,20,12,1,2,166.66
3,Namibia Vs Sri Lanka,Namibia Innings,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33
4,Namibia Vs Sri Lanka,Namibia Innings,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33


In [37]:
#format the column "team_innings"
df_batsman_sum['team_innings'] = df_batsman_sum['team_innings'].apply(lambda x: x.replace(' Innings', ''))

In [38]:
#rename column names

df_batsman_sum.rename({'ball': 'balls'}, axis = 1, inplace = True)
df_batsman_sum.rename({'run': 'runs'}, axis = 1, inplace = True)

In [39]:
#creating a new column out/notout 

df_batsman_sum['out/not_out'] = df_batsman_sum['dismissal'].apply(lambda x: "out" if (len(x) > 0) else "not_out")

**Get the Match_ID**

In [42]:
df_batsman_sum['match_id'] = df_batsman_sum['match'].map(match_ids_dict)

In [43]:
#reforming the dataframe and finally saving it

df_batsman_sum = df_batsman_sum[['match_id', 'match', 'team_innings', 'batting_pos', 'batsman_name', 'out/not_out', 'runs', 'balls',
       '4s', '6s', 'SR']]
df_batsman_sum.head(15)

,match_id,match,team_innings,batting_pos,batsman_name,out/not_out,runs,balls,4s,6s,SR
0,T20I # 1823,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,out,3,6,0,0,50.00
1,T20I # 1823,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,out,9,9,1,0,100.00
2,T20I # 1823,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,out,20,12,1,2,166.66
3,T20I # 1823,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,out,26,24,2,0,108.33
4,T20I # 1823,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),out,20,24,0,0,83.33
5,T20I # 1823,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,out,44,28,4,0,157.14
6,T20I # 1823,Namibia Vs Sri Lanka,Namibia,7,David Wiese,out,0,1,0,0,0.00
7,T20I # 1823,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,not_out,31,16,2,2,193.75
8,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,out,9,10,1,0,90.00
9,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,out,6,6,0,0,100.00


In [44]:
# we find some unusal string in the batsman names, let's clean/replace them

df_batsman_sum['batsman_name'] = df_batsman_sum['batsman_name'].apply(lambda x: x.replace('â€', ''))

In [45]:
df_batsman_sum.head(15)

,match_id,match,team_innings,batting_pos,batsman_name,out/not_out,runs,balls,4s,6s,SR
0,T20I # 1823,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,out,3,6,0,0,50.00
1,T20I # 1823,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,out,9,9,1,0,100.00
2,T20I # 1823,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,out,20,12,1,2,166.66
3,T20I # 1823,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,out,26,24,2,0,108.33
4,T20I # 1823,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),out,20,24,0,0,83.33
5,T20I # 1823,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,out,44,28,4,0,157.14
6,T20I # 1823,Namibia Vs Sri Lanka,Namibia,7,David Wiese,out,0,1,0,0,0.00
7,T20I # 1823,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,not_out,31,16,2,2,193.75
8,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,out,9,10,1,0,90.00
9,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendis,out,6,6,0,0,100.00


In [49]:
# https://stackoverflow.com/questions/71588833/%C3%82-always-at-beginning-of-string-when-stored-to-dataframe-csv
#saving the dataframe

df_batsman_sum.to_csv('t20_csv_files/fact_batsman_summary.csv', index = False, encoding="cp1252")

### **create csv file: fact_bowling_summary**

In [51]:
#store all the json objects in a list
with open('t20_json_files/t20_wc_bowling_summary.json') as f:
  data = json.load(f)
  all_json_records = []
  for json_obj in data:
    all_json_records.extend(json_obj["bowlingSummary"])
  
# print(all_json_records)

In [52]:
df_bowling_sum = pd.DataFrame(all_json_records)
print(df_bowling_sum.shape)
df_bowling_sum.head()

(500, 13)


,match,bowling_team,bowler_name,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,no_balls
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0


**Add Match_IDS**

In [53]:
df_bowling_sum['match_id'] = df_bowling_sum['match'].map(match_ids_dict)

In [54]:
df_bowling_sum.head()

,match,bowling_team,bowler_name,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,no_balls,match_id
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823


In [55]:
df_bowling_sum.columns

Index(['match', 'bowling_team', 'bowler_name', 'overs', 'maiden', 'runs',
       'wickets', 'economy', '0s', '4s', '6s', 'wides', 'no_balls',
       'match_id'],
      dtype='object')

In [56]:
#reforming the dataframe and finally saving it

df_bowling_sum = df_bowling_sum[['match_id', 'match', 'bowling_team', 'bowler_name', 'overs', 'maiden', 'runs',
       'wickets', 'economy', '0s', '4s', '6s', 'wides', 'no_balls']]
df_bowling_sum.head(15)

,match_id,match,bowling_team,bowler_name,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,no_balls
0,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0
4,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0
5,T20I # 1823,Namibia Vs Sri Lanka,Namibia,Gerhard Erasmus,1,0,8,0,8.00,1,1,0,0,0
6,T20I # 1823,Namibia Vs Sri Lanka,Namibia,David Wiese,4,0,16,2,4.00,13,1,0,0,0
7,T20I # 1823,Namibia Vs Sri Lanka,Namibia,Bernard Scholtz,4,0,18,2,4.50,10,1,0,0,0
8,T20I # 1823,Namibia Vs Sri Lanka,Namibia,Ben Shikongo,3,1,22,2,7.33,6,3,0,0,0
9,T20I # 1823,Namibia Vs Sri Lanka,Namibia,JJ Smit,3,0,16,1,5.33,7,0,0,1,0


In [59]:
#saving this dataframe finally
df_bowling_sum.to_csv('t20_csv_files/fact_bowling_summary.csv', index = False)

### **create csv file: dim_players**

In [65]:
import json
with open('t20_json_files/t20_wc_player_info.json') as f:
  data = json.load(f)
# print(data)

In [73]:
#create the datafranme

df_players = pd.DataFrame(data)

print(df_players.shape)
df_players.head()

(219, 6)


,name,team,Batting_Style,Bowling_Style,Playing_Role,description
0,Hazratullah Zazai,Afghanistan,Left hand Bat,Slow Left arm Orthodox,Opening Batter,Afghanistan's ascent up the global rankings fr...
1,Rahmanullah Gurbazâ€,Afghanistan,Right hand Bat,,Wicketkeeper Batter,
2,Ibrahim Zadran,Afghanistan,Right hand Bat,Right arm Medium fast,Opening Batter,
3,Usman Ghani,Afghanistan,Right hand Bat,,Opening Batter,One of the most exciting new talents to emerge...
4,Najibullah Zadran,Afghanistan,Left hand Bat,Right arm Offbreak,Middle order Batter,Najibullah Zadran packs a punch with his belli...


In [74]:
#cleaning the name

dim_players['name'] = dim_players['name'].apply(lambda x: x.replace('â€', ''))
dim_players['name'] = dim_players['name'].apply(lambda x: x.replace('†', ''))

In [75]:
df_players[df_players['team'] == 'India']

,name,team,Batting_Style,Bowling_Style,Playing_Role,description
166,KL Rahul,India,Right hand Bat,,Opening Batter,"A tall, elegant right-hand batsman who can kee..."
167,Rohit Sharma(c),India,Right hand Bat,Right arm Offbreak,Top order Batter,"Languid and easy on the eye, Rohit Sharma owne..."
168,Virat Kohli,India,Right hand Bat,Right arm Medium,Top order Batter,India has given to the world many a great cric...
169,Suryakumar Yadav,India,Right hand Bat,"Right arm Medium, Right arm Offbreak",Batter,Hard-hitting 360-degree batter Suryakumar Yada...
170,Bhuvneshwar Kumar,India,Right hand Bat,Right arm Medium,Bowler,"At the time of his India debut in 2012, Bhuvne..."
171,Arshdeep Singh,India,Left hand Bat,Left arm Medium fast,Bowler,
172,Mohammed Shami,India,Right hand Bat,Right arm Fast,Bowler,Mohammed Shami was India's leading fast bowler...
173,Axar Patel,India,Left hand Bat,Slow Left arm Orthodox,Bowling Allrounder,Left-arm spinner Axar Patel has been increasin...
174,Hardik Pandya,India,Right hand Bat,Right arm Medium fast,Allrounder,Hardik Pandya swears by living life king size ...
175,Ravichandran Ashwin,India,Right hand Bat,Right arm Offbreak,Bowling Allrounder,R Ashwin took the tricks and skills he learned...


In [76]:
df_players.to_csv('t20_csv_files/dim_players.csv', index = False)